
# Pipeline de Otimização de Hiperparâmetros para Regressão

Este notebook implementa um pipeline completo de otimização de hiperparâmetros com validação cruzada para múltiplos algoritmos de regressão. Utilizamos `BayesSearchCV` do pacote `scikit-optimize` para encontrar os melhores parâmetros de cada modelo.

## Etapas:
- Divisão dos dados em `dev` e `test_final`
- Construção de pipelines com pré-processamento
- Otimização de hiperparâmetros com validação cruzada
- Avaliação com métricas: R², RMSE, RME
- Visualização dos resultados com boxplot

## Parametrização:
- `SEED`: controle de aleatoriedade
- `CV_FOLDS`: número de folds na validação cruzada
- `SPLIT`: proporção do conjunto de teste final
- `METRIC_TO_OPT`: métrica principal para otimização

## Requisitos:
- Arquivo externo `ALGO_CONFIGS.json` com metadados dos algoritmos


In [1]:

import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

import importlib

# Parametrização
SEED = 42
CV_FOLDS = 5
SPLIT = 0.2
METRIC_TO_OPT = 'r2'


In [2]:

def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

def rme(y_true, y_pred):
    return np.mean((y_true - y_pred) / y_true)


In [3]:

# Carregar configurações dos algoritmos
algo_configs_path = "ALGO_CONFIGS.json"
if not os.path.exists(algo_configs_path):
    raise FileNotFoundError("Arquivo ALGO_CONFIGS.json não encontrado.")

with open(algo_configs_path, "r") as f:
    algo_configs = json.load(f)


In [4]:

from sklearn.datasets import make_regression
X, y = make_regression(n_samples=1000, n_features=20, noise=0.1, random_state=SEED)

# Divisão dos dados
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=SPLIT, random_state=SEED)


In [5]:

cv_results = {}
test_results = {}

for algo_name, config in algo_configs.items():
    print(f"Processando: {algo_name}")

    module = importlib.import_module(config["module_name"])
    model_class = getattr(module, config["model_class"])
    model = model_class(**config["default_params"])

    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', model)
    ])

    search_space = {f'regressor__{k}': v for k, v in config["search_space"].items()}

    opt = BayesSearchCV(
        estimator=pipe,
        search_spaces=search_space,
        n_iter=20,
        cv=CV_FOLDS,
        scoring=METRIC_TO_OPT,
        random_state=SEED,
        return_train_score=False,
        n_jobs=-1
    )

    opt.fit(X_dev, y_dev)
    y_pred_test = opt.predict(X_test)

    test_metrics = {
        'r2': r2_score(y_test, y_pred_test),
        'rmse': rmse(y_test, y_pred_test),
        'rme': rme(y_test, y_pred_test)
    }
    test_results[algo_name] = test_metrics

    fold_metrics = {'r2': [], 'rmse': [], 'rme': []}
    for train_idx, val_idx in opt.cv.split(X_dev, y_dev):
        X_train, X_val = X_dev[train_idx], X_dev[val_idx]
        y_train, y_val = y_dev[train_idx], y_dev[val_idx]

        opt.estimator.fit(X_train, y_train)
        y_pred_val = opt.estimator.predict(X_val)

        fold_metrics['r2'].append(r2_score(y_val, y_pred_val))
        fold_metrics['rmse'].append(rmse(y_val, y_pred_val))
        fold_metrics['rme'].append(rme(y_val, y_pred_val))

    cv_results[algo_name] = fold_metrics


Processando: LinearRegression


KeyError: 'module_name'

In [ ]:

metrics_df = []
for algo, metrics in cv_results.items():
    for i in range(CV_FOLDS):
        metrics_df.append({
            'Algorithm': algo,
            'Fold': i+1,
            'R2': metrics['r2'][i],
            'RMSE': metrics['rmse'][i],
            'RME': metrics['rme'][i]
        })

metrics_df = pd.DataFrame(metrics_df)

plt.figure(figsize=(15, 5))
sns.boxplot(x='Algorithm', y=METRIC_TO_OPT.upper(), data=metrics_df.rename(columns={METRIC_TO_OPT.upper(): METRIC_TO_OPT}))

for i, algo in enumerate(test_results):
    plt.scatter(i, test_results[algo][METRIC_TO_OPT], color='red', label='Test Final' if i == 0 else "")

plt.title(f"Boxplot da métrica {METRIC_TO_OPT.upper()} por algoritmo (CV) com Test Final")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
